In [18]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
from datetime import datetime, timedelta


import ast
import json

Import data

In [21]:
sked = pd.read_csv("~/Desktop/STUDBOT/schedule_no_inter.csv")
sked['datetime_gmt'] = pd.to_datetime(sked['datetime_gmt']).apply(lambda x: x.strftime('%Y-%m-%dT%H:%M'))


Round up the time

In [22]:
sked

,Unnamed: 0,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,...,referee,stadium_id,stadium,datetime_et,datetime_gmt,latitude,longitude,indoor,turf,elevation
0,1,2021_01_DAL_TB,2021,REG,1,2021-09-09,Thursday,20:20,DAL,29.0,...,Shawn Hochuli,TAM00,Raymond James Stadium,2021-09-09 21:00:00,2021-09-10T01:00:00,27.975967,-82.503350,No,Grass,11
1,2,2021_01_PHI_ATL,2021,REG,1,2021-09-12,Sunday,13:00,PHI,32.0,...,Scott Novak,ATL97,Mercedes-Benz Stadium,2021-09-12 13:00:00,2021-09-12T17:00:00,33.755489,-84.401993,Retractable,Turf,314
2,3,2021_01_PIT_BUF,2021,REG,1,2021-09-12,Sunday,13:00,PIT,23.0,...,John Hussey,BUF00,New Era Field,2021-09-12 13:00:00,2021-09-12T17:00:00,42.773739,-78.786978,No,Turf,222
3,4,2021_01_NYJ_CAR,2021,REG,1,2021-09-12,Sunday,13:00,NYJ,14.0,...,Clay Martin,CAR00,Bank of America Stadium,2021-09-12 13:00:00,2021-09-12T17:00:00,35.225808,-80.852861,No,Turf,220
4,5,2021_01_MIN_CIN,2021,REG,1,2021-09-12,Sunday,13:00,MIN,24.0,...,Adrian Hill,CIN00,Paul Brown Stadium,2021-09-12 13:00:00,2021-09-12T17:00:00,39.095442,-84.516039,No,Turf,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1112,1113,2024_18_MIA_NYJ,2024,REG,18,2025-01-05,Sunday,13:00,MIA,NaN,...,NaN,NYC01,MetLife Stadium,2025-01-05 13:00:00,2025-01-05T18:00:00,40.812194,-74.076983,No,Turf,2
1113,1114,2024_18_NYG_PHI,2024,REG,18,2025-01-05,Sunday,13:00,NYG,NaN,...,NaN,PHI00,Lincoln Financial Field,2025-01-05 13:00:00,2025-01-05T18:00:00,39.900775,-75.167453,No,Hybrid,4
1114,1115,2024_18_CIN_PIT,2024,REG,18,2025-01-05,Sunday,13:00,CIN,NaN,...,NaN,PIT00,Acrisure Stadium,2025-01-05 13:00:00,2025-01-05T18:00:00,40.446786,-80.015761,No,Grass,229
1115,1116,2024_18_NO_TB,2024,REG,18,2025-01-05,Sunday,13:00,NO,NaN,...,NaN,TAM00,Raymond James Stadium,2025-01-05 13:00:00,2025-01-05T18:00:00,27.975967,-82.503350,No,Grass,11


Weather data pull

In [23]:
sked['weather_info'] = None

for i in tqdm(range(len(sked)), desc = 'Progress'):
    response = requests.get(f'https://archive-api.open-meteo.com/v1/archive?latitude={sked["latitude"].values[i]}&longitude={sked["longitude"].values[i]}&start_date={pd.to_datetime(sked["datetime_gmt"].values[i]).date()}&end_date={pd.to_datetime(sked["datetime_gmt"].values[i]).date()}&timezone=EST&temperature_unit=fahrenheit&hourly=temperature_2m,apparent_temperature,precipitation,rain,snowfall,weathercode,windspeed_10m,winddirection_10m,windgusts_10m')
    sked['weather_info'].values[i] = response.json()

Progress: 100%|██████████| 1117/1117 [09:07<00:00,  2.04it/s]


Dataframe the weather for each game

In [61]:
def extract_data(row):
    if row['indoor'] == 'No':
        try: 
            json_data =  row['weather_info']
            #start_date = str(row['datetime_gmt'].strftime('%Y-%m-%d')) + str(row['Round Time'].strftime('T%H:%M'))
            start_date = row['datetime_gmt']
            # Find the index where 'time' matches 'roundup_start_date'
            time_index = json_data["hourly"]["time"].index(start_date)

            # Extract data based on the index
            temperature = json_data["hourly"]["temperature_2m"][time_index]
            precipitation = np.sum(json_data["hourly"]["precipitation"][time_index:time_index+3])
            windspeed = json_data["hourly"]["windspeed_10m"][time_index]
            windgusts = json_data["hourly"]["windgusts_10m"][time_index]
        
            return pd.Series({'temperature': temperature, 'precipitation': precipitation, 'windspeed': windspeed, 'windgusts': windgusts})
        except:
            return None
    else: 
        json_data = row['weather_info']
        
        return pd.Series({ 'temperature': 70, 'precipitation': 0, 'windspeed': 0, 'windgusts': 2.5})

new_col = sked.apply(extract_data, axis=1)
test = pd.concat([sked, new_col], axis=1)

#sked.to_csv('final_schedule.csv', index=False)

Merge kicks with schedule by teams and date

In [60]:
test.to_csv("~/Desktop/STUDBOT/noninter_final.csv")